# Interactive Monte-Carlo Tree Search for poetry generation 

This notebooks shows how enact can be used to instantiate a generic monte-carlo
tree search over a simple line-by-line poetry generator. The result is a search
process that spends more time exploring promising poetic direction.

The last section shows how user input can be injected during MCTS, in order to
allow a user to dynamically provide score prompt examples for generated poetry.

## Prerequisites and API keys

In [1]:
%pip install enact --quiet
%pip install openai --quiet
%pip install randomname --quiet
%pip install tqdm --quiet


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re

import enact       # Journaled executions
import openai      # GPT API
import randomname  # Extra randomness for prompt.
import tqdm        # progress bar

import common      # API keys


# Read from file or env variable.
openai.api_key = common.OPENAI_API_KEY.get()

## Defining a line-by-line poetry generator

We first define a line-by-line poetry generator by calling GPT in a for-loop.

In [3]:
class Failed(enact.ExceptionResource):
  """Raised when a subtask fails."""


@enact.register  # Track this function's execution.
def gpt(prompt: str, temperature: float=1.0, model='gpt-4') -> str:
  """Call the GPT completion API"""
  response = openai.ChatCompletion.create(
    model=model,
    messages=[{'role': 'system', 'content': prompt}],
    temperature=temperature)
  return response['choices'][0]['message']['content']

@enact.register
def extend_poem(
    poem: str, prompt: str, lines: int,
    temperature: float = 1.0) -> str:
  """Extend the poem by one line."""
  line_index = len(poem.strip().split('\n'))
  gpt_prompt = f'''
We are writing a poem for the prompt: {prompt}.

This is the poem so far:

{poem if poem else '<NEW POEM>'}

Extend the poem by one single line. Respond only with the new line and say
nothing else. This will be line nr {line_index + 1} out of a total of {lines}
lines.

Avoid cliches.

Respond only with one line.

Ignore the following random seed words:
{', '.join(randomname.generate('nouns/') for _ in range(10))}
'''
  result = gpt(gpt_prompt, temperature, model='gpt-4')
  lines = result.strip().split('\n')
  if len(lines) != 1:
    raise Failed('Responded with more than one line')
  line: str = lines[0]
  line = line.strip('"\'').strip()
  return poem + f'\n{line}'

@enact.register
def generate_poem(prompt: str, lines: int) -> str:
  """Generate a poem line by line."""
  poem = ''
  for _ in range(lines):
    poem = extend_poem(poem, prompt, lines)
  return poem


Our poetry generator can be run as follows, although it may sometimes fail,
e.g., when GPT produces output in a wrong format.

In [4]:
try:
  two_line_poem = generate_poem(
    'a two line poem inspired by the cure song "a forest"', lines=2)
  print(two_line_poem)
except Exception as e:
  print(f'Failed with "{e}". Try again.')


Through the misty black, echoes the silence of our heart's lost track.
In the swallowing darkness, we find truth amongst the trees, unseen yet intact.


## Define a critic

We would like to search the space of possible executions of our `generate_poem`
function. In order to do so, we need to define a scoring function that can
assign a numeric value to generated poems. We use another call to GPT for this.

In [5]:
DEFAULT_EXAMPLES ="""
Example 1:

"Do not go gentle into that good night,
Old age should burn and rave at close of day;
Rage, rage against the dying of the light.

Though wise men at their end know dark is right,
Because their words had forked no lightning they
Do not go gentle into that good night."

SCORE: 10.0

Example 2:

"To fling my arms wide
In some place of the sun,
To whirl and to dance
Till the white day is done.
Then rest at cool evening
Beneath a tall tree
While night comes on gently,
    Dark like me—
That is my dream!"

SCORE: 8.3

Example 3:

"A winter wonderland, draped in pure white.
The snowflakes dance, in the pale moonlight.
Shimmering crystals, glistening so bright.
Blankets of snow, whispering secrets of the night."

SCORE: 3.1

"""

@enact.register
def score_poem(poem: str, examples: str=DEFAULT_EXAMPLES) -> float:
  gpt_prompt: str = f'''
You are a sophisticated poetry critic. Score the following poem on a scale from
0 to 10, where 0 is the worst poem and 10 is the best poem. You are a harsh 
critic and hate awkward, tryhard phrasings. Scores from 4-6 are decent poem
written by a english lit major. 7 and above are publishable quality. Use 8 and 9
for excellent poems. A score of ten is reserved for masterpieces. You pay
particular attention to the rhythm of a poem.

{examples}

Respond in the following format:

<short summary of positive and negatives. no longer than a line>
SCORE: <score>

This is the poem:
{poem}
'''
  score = gpt(gpt_prompt, 1.1)
  return float(re.search(r'SCORE: ([\d\.]+)', score).group(1))


We can apply our poetry critic to the poem we generated earlier:

In [6]:
try:
  print(two_line_poem)
  print(f'Score: {score_poem(two_line_poem)}')
except Exception as e:
  print(f'Try again. Generation failed with: {e}')


Through the misty black, echoes the silence of our heart's lost track.
In the swallowing darkness, we find truth amongst the trees, unseen yet intact.
Score: 4.5


## Implementing Monte Carlo Tree Search

Enact can rewind and replay executions of registered python functions.  We can
use this to explore the space of possible program executions using Monte Carlo
Tree Search.


In [13]:
import dataclasses
from typing import Callable, Dict, List, Optional
import numpy as np


# Increase this number for more exploration
EXPLORATION_AMOUNT = 5
# Increase this to explore more from the root node,
# i.e., to create more new poems from scratch.
EXPLORE_ROOT_NODE = 8
# Conservative quality prior to add for bootstrapped LCB.
LCB_PRIOR = [1.0]


@enact.register
@dataclasses.dataclass
class Node(enact.Resource):
  invocation: enact.Invocation  # A (possibly-partial) execution.
  parent: Optional['Node']      # The parent node.
  scores: List[float]           # List of scores for this subtree.

  @property
  def tree_ucb(self) -> float:
    """Computes a heuristic UCB on scores for tree search."""
    if not self.parent:
      parent_scores = EXPLORE_ROOT_NODE * len(self.scores)
    else:
      parent_scores = len(self.parent.scores)
    return np.mean(self.scores) + EXPLORATION_AMOUNT * np.sqrt(
      np.log(1e-2 + parent_scores) / len(self.scores))

  @property
  def lcb(self) -> float:
    """Computes a lower confidence bound on scores."""
    scores = self.scores
    scores = scores + LCB_PRIOR
      
    return  np.percentile(
      np.mean(np.random.choice(
        scores, (10000, len(scores))), axis=1),
      30.0)


@enact.register
@dataclasses.dataclass
class MCTS(enact.Resource):
  scorer: Callable[[str], float]
  nodes: Dict[str, Node] = dataclasses.field(
    default_factory=dict)

  def add(
      self,
      invocation: enact.Invocation):
    """Add an invocation to the MCTS tree.""" 
    child = None
    score = self.compute_score(invocation)
    if not score:
      raise Failed('Could not compute score.')
    while True:
      cur = enact.commit(invocation)
      node = self.nodes.setdefault(cur.id, Node(invocation, None, []))
      node.scores.append(score)
      if child:
        child.parent = node
      child = node
      if not invocation.response().children:
        break
      invocation = invocation.rewind()  # Generate execution prefix.
    return score

  def next_node(self):
    """Select next node according to highest UCB."""
    idx = np.argmax([node.tree_ucb for node in self.nodes.values()])
    return list(self.nodes.values())[idx]

  def compute_score(self, invocation: enact.Invocation) -> Optional[float]:
    """Assign a score to the invocation."""
    if not invocation.successful():
      return -1.0
    return self.scorer(invocation.get_output())
  
  @enact.register
  def step(self):
    """Run one step of MCTS and return generated output and score."""
    assert self.nodes, 'Please call add before step.'
    node = self.next_node()
    if node.invocation.response().children:
      print(f'\nContinuing partial poem:\n{node.invocation.response().children[-1]().get_output()}')
    else:
      print(f'\nCreating new root poem')
    invocation = self.replay(node.invocation)
    score = self.add(invocation)
    if invocation.successful():
      return invocation.get_output(), score

  @enact.register
  def invoke(self, fun: Callable, args=(), kwargs=None):
    """Wrap invocation in a registered function, since it is non-deterministic."""
    return enact.invoke(fun, args=args, kwargs=kwargs)

  @enact.register
  def replay(self, invocation: enact.Invocation) -> enact.Invocation:
    """Wrap replay in a registered function, since it is non-deterministic."""
    return invocation.replay()

  @enact.register
  def add_seed(self, fun, args=(), kwargs=None):
    """Add a seed invocation. Return success."""
    invocation = self.invoke(fun, args=args, kwargs=kwargs)
    if not invocation.successful():
      return False
    try:
      score = self.add(invocation)
    except Failed:
      return False
    return score is not None
  
  @enact.register
  def run(self, fun, args=(), kwargs=None, iterations=100):
    """Run MCTS for a given number of iterations."""
    for _ in range(10):
      if self.add_seed(fun, args=args, kwargs=kwargs):
        break
    else:
      raise Failed('Could not find a successful seed invocation.')
      
    for _ in tqdm.trange(iterations):
      try:
        output, score = self.step()
        if output:
          print(output)
          print(f'Score: {score}')
      except Failed:
        pass

  def highest_lcb(self):
    """Return the highest LCB output."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: node.lcb).invocation.get_output()

  def highest_mean(self):
    """Return the output with the higest mean score."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: np.mean(node.scores)).invocation.get_output()

Enact requires a store object in which calltraces (including inputs and outputs)
are stored.

In [14]:
store = enact.InMemoryStore()

We can now run MCTS for a set number of iterations using our automatic scoring.

In [16]:
with store:
  mcts = MCTS(score_poem)
  mcts.run(
    generate_poem, 
    args=('A poem inspired by the cure song "A forest" '
          'in the style of Robert Frost', 4),
    iterations=10)
  print(f'\nHighest LCB:\n{mcts.highest_lcb()}')
  print(f'\nHighest mean:\n{mcts.highest_mean()}')

  0%|          | 0/10 [00:00<?, ?it/s]


Creating new root poem


 10%|█         | 1/10 [00:09<01:23,  9.31s/it]


Through twisted branches, age-old remembrance of song
Creaking under the burden of time, silent whispers among the moss.
In shadows deep, where paths meander lost, under starless cloth embossed.
I walk, ensnared in twilight's mire, chasing echoes of your ghost.
Score: 4.7

Creating new root poem


 20%|██        | 2/10 [00:20<01:22, 10.35s/it]

...


Through twisted branches, age-old remembrance of song
Creaking under the burden of time, silent whispers among the moss.
Echoing footfall in the hollowed demesne, so stark in its loss.
A spectral presence whispered through the leaves, guard of the haunted dark.
Score: 4.5

Continuing partial poem:

Where silhouettes of trees, do silent ghosts entwine,


 90%|█████████ | 9/10 [01:08<00:06,  6.85s/it]


Where silhouettes of trees, do silent ghosts entwine,
In shadows deep and whispers of an echoing song, divine.
Between the ancient grains of time, my footsteps tread yet find no sign.
Of mysteries untold, that blooms in the enigma of night, confined.
Score: 4.9

Continuing partial poem:

Where silhouettes of trees, do silent ghosts entwine,
In the hushed whispers of the wind, secrets are assigned.


100%|██████████| 10/10 [01:14<00:00,  7.47s/it]


Where silhouettes of trees, do silent ghosts entwine,
In the hushed whispers of the wind, secrets are assigned.
Beneath the moonlight’s dappled shroud, in icicles faith is confined.
Where shadows reign and time is consigned.
Score: 5.6

Highest LCB:

Through twisted branches, age-old remembrance of song
Underneath the veil of unraveling fog, the moon hums along.
When the stars bloom, whispers dare to dart between the pine bark throng.
Silken wind threads into lullabies, as it rustles the cradle of the night's strong.

Highest mean:

Through twisted branches, age-old remembrance of song
Underneath the veil of unraveling fog, the moon hums along.
When the stars bloom, whispers dare to dart between the pine bark throng.
Silken wind threads into lullabies, as it rustles the cradle of the night's strong.


## Hybrid user / machine critic

Above, we showed how the `generate_poem` function can be boosted using MCTS and
a scoring function. The remainder of this notebook shows how we can inject human
input into the scoring function.

We define an adaptive scoring callable, that will query the user 3 times for a
score, before extrapolating from the user scores autonomously by adding the
user score examples to the prompt for a GPT scorer.

In [17]:
@enact.register
@dataclasses.dataclass
class TrainingExample(enact.Resource):
  output: str
  score: float

@enact.register
@dataclasses.dataclass
class TrainableScorer(enact.Invokable):
  examples_to_collect: int = 3
  examples: List[TrainingExample] = dataclasses.field(default_factory=list)
  
  def call(self, poem: str) -> float:
    """Score a poem."""
    if len(self.examples) < self.examples_to_collect:
      # Sample an example from the user.
      score = float(enact.request_input(
        requested_type=str,
        for_value=poem,
        context='Please score the poem from 1 to 10'))
      self.examples.append(TrainingExample(poem, score))
      return score
    # Once enough examples are collected, prompt GPT with the examples.
    examples = '\n\n'.join(
      f'Example {i}:\n{example.output}\nSCORE:{example.score}'
      for i, example in enumerate(self.examples))
    return score_poem(poem, examples)


@enact.register
def generate_poem_with_user_training(prompt: str) -> str:
  mcts = MCTS(TrainableScorer())
  mcts.run(generate_poem, args=(prompt, 4), iterations=15)
  return mcts.highest_mean()


We use `enact.InvocationGenerator` to step through all user inputs that are
generated during the invocation:

In [19]:
import sys

with store:
  inv_gen = enact.InvocationGenerator.from_callable(
    generate_poem_with_user_training,
    ('a poem inspired by the cure song "A forest"',))
  for input_request in inv_gen:
    print(f'\n{input_request.context}:\n{input_request.for_value()}')
    sys.stdout.flush()
    user_input = input()
    print(f'received user input: {user_input}')
    inv_gen.set_input(user_input)
  print(f'\nResult:\n{inv_gen.invocation.get_output()}')
    


Please score the poem from 1 to 10:

Where whispers float 'neath a masquerade of leaves,
In twilight’s song, shadows unfurl their solemn creeds.
Among the solemn giants, the distant echo of love bleeds.
A fleeting solace where time and silence interweave.
received user input: 3.8


  0%|          | 0/15 [00:00<?, ?it/s]


Creating new root poem


  0%|          | 0/15 [00:06<?, ?it/s]


Please score the poem from 1 to 10:

Echoes of your voice are shadows dancing with the moon.
Lost in the labyrinth of pines, the veil between us thins.
The whispering wind weaves tales of love once painted in emerald green.
Your silhouette shivers in the serenade of the silver-lit wilderness.


...


Echoes of your voice are shadows dancing with the moon.
Lost in the labyrinth of pines, the veil between us thins.
Chasing phantoms through dew-kissed ferns, to the rhythm of our sins.
Whispers of our past lie buried beneath moss-kissed stones.
Score: 5.4

Continuing partial poem:

Echoes of your voice are shadows dancing with the moon.
In the quiet whisper, haunting echoes from buried roots in tune.


 93%|█████████▎| 14/15 [01:09<00:06,  6.05s/it]


Echoes of your voice are shadows dancing with the moon.
In the quiet whisper, haunting echoes from buried roots in tune.
Through a veil of mist, you're an illusion in the evergreen gloom.
A melody etched in bark, looms in the silence that's deafeningly luminous.
Score: 4.7

Continuing partial poem:

Echoes of your voice are shadows dancing with the moon.
In the somber ballet, trees whisper tales of long lost truth.


100%|██████████| 15/15 [01:14<00:00,  5.00s/it]


Echoes of your voice are shadows dancing with the moon.
In the somber ballet, trees whisper tales of long lost truth.
Lamenting leaves fall, a symphony of silent sorrow.
Beneath the solemn sky, the forest veils its mysteries in nocturnal serenade.
Score: 5.4

Result:

Echoes of your voice are shadows dancing with the moon.
In the somber ballet, trees whisper tales of long lost truth.
Beneath the velvet skies, the melody of dew-kissed leaves unfold.
Yet, in that orchestra of darkness, the forest sighs your phantom tune.
